### Tutorial D2 (Built-in Tools): TOMTOM

`FIMO` is a tool for taking a set of PWMs and scanning them against a set of one-hot encoded sequences; in contrast, `TOMTOM` is a tool for comparing pairs of PWMs to determine their similarity. Classically, tools like `TOMTOM` are used to identify redundancies in motif databases and to estimate similarities between binding profiles of proteins. However, `TOMTOM` is also frequently used by researchers who see motifs that are highlighted by some method (such as attributions from machine learning models) and need to know what that motif corresponds to. In this manner, `TOMTOM` serves as a useful automatic annotation tool for discoveries made using `tangermeme`.

Although at first glance `FIMO` and `TOMTOM` appear similar because they involve comparing motifs with something to identify statistical similarities, the difference between operating on long one-hot encoded sequences and a second motif require large methodological changes. First, calculating a background distribution for `FIMO` can be done exactly because you know that the sequences being scanned are limited to being one-hot encoded, whereas the background distribution for `TOMTOM` must be empirically calculated from the set of provided motifs. Second, because motifs are much shorter than the sequences usually being scanned by `FIMO`, the best alignment between two motifs likely involve some amount of overhang on either end of the alignment. Calculating scores and p-values in a manner that don't automatically undervalue imperfect alignments is non-trivial. 

For more information about `TOMTOM`, I'd suggest reading [the original paper](https://genomebiology.biomedcentral.com/articles/10.1186/gb-2007-8-2-r24) and the [subsequent follow-up work](https://pubmed.ncbi.nlm.nih.gov/21543443/) that improves the calculation of p-values for alignments with overhangs.

#### Using TOMTOM

One can run the `TOMTOM` algorithm by calling the `tomtom` function on a pair of lists, where both lists are PWMs that are each of shape `(len(alphabet), motif_length)`. The background distribution is built primarily using the targets, so make sure that the first list is the queries you want to evaluate and the second list is the target distribution you want to match against. For example, if you want to score a single one-hot encoded sequence, you can do the following:

In [1]:
import numpy

from tangermeme.io import read_meme
from tangermeme.utils import one_hot_encode

from tangermeme.tools.tomtom import tomtom

q = [one_hot_encode("ACGTGT").double()]

targets = read_meme("JASPAR2020_CORE_non-redundant_pfms_meme.txt")
target_names = numpy.array([name for name in targets.keys()])
target_pwms = [pwm for pwm in targets.values()]


p, scores, offsets, overlaps, strands = tomtom(q, target_pwms)
p.shape

(1, 1646)

You will get several properties of the best alignment between the provided queries and each of the targets. Here, because there was only one query, the first dimension of each returned tensor only has one elements.

If we want to get the names of the statistically significant matches we can easily cross-reference the p-values with the names in the target database.

In [2]:
target_names[p[0] < 0.0001]

array(['MA0310.1 HAC1', 'MA0622.1 Mlxip', 'MA0930.1 ABF3',
       'MA0931.1 ABI5', 'MA1033.1 OJ1058_F05.8', 'MA1331.1 BEH4',
       'MA1332.1 BEH2', 'MA1333.1 BEH3', 'MA1493.1 HES6'], dtype='<U28')

The returned scores are the integerized score of the best alignment, the offets indicate where the best alignment starts for the query, the overlaps are the number of positions that overlap between the target and the query, and the strand is whether the best alignment is on the forward (0) strand or the negative (1) strand.

Sometimes, you will be comparing one set of PWMs against a second set of PWMs though, rather than a single value versus a set of PWMs. Doing so is pretty much the same as before, except that the list of query motifs has more than one value in it. We can demonstrate that by comparing the test set of motifs used in the unit tests against the full set of motifs.

In [3]:
query = list(read_meme("../../tests/data/test.meme").values())

p, scores, offsets, overlaps, strands = tomtom(query, target_pwms)
p.shape

(12, 1646)

The set of motifs used in the unit tests are a few randomly selected motifs from the JASPAR set, so the p-values should indicate perfect matches.

In [4]:
p.min(axis=-1)

array([-3.17523785e-13, -5.60440583e-13,  5.38370792e-09, -9.46798195e-13,
       -2.70006240e-13,  5.71406977e-07,  4.77395901e-14, -2.93542968e-13,
       -2.18491891e-13,  2.02698924e-09, -1.84918747e-12,  1.90786831e-09])

#### Timings

When one uses the MEME suite webserver to run TOMTOM, they may notice that it takes several seconds to run a single query against a target database. Although the command-line tool is faster (as jobs do not need to be submitted, queued, and then wait for available resources), one may still wonder if it is possible to do large-scale comparisons with it. 

The implementation of `TOMTOM` in `tangermeme` scales significantly better than the command-line tool (though you might notice it takes a few seconds for numba to load initially due to some issues there). There are several reasons for this but perhaps the most notable are built in multithreading through numba and caching null distributions for each target length, rather than having to recalculate them for each query-target pair. These efficiencies significantly speed up the implementation in `tangerememe` without compromising on the exactness. However, importantly, other speed improvements (approximate medians, binning similar targets) further speed up the algorithm but do slightly alter the results. In practice, there is very little observed difference, but this is why you may not see identical results.

Let's do a quick timing test comparing the 1646 motifs in the JASPAR set against themselves. This involves doing 1646x1646x2 comparisons, with the x2 being because we are also doing reverse complement comparisons.

In [5]:
%timeit tomtom(target_pwms, target_pwms)

1.06 s ± 56.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Just one second to calculate the complete similarity matrix -- including allocating the intermediary memory and spinning up the threadpool.

How long would it take to do a much larger calculation? We can estimate that by copying the array 10 times in both directions.

In [6]:
%timeit tomtom(target_pwms*10, target_pwms*10)

9.09 s ± 121 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Looks like it only takes ~10s, which is 10x faster than you might expect, since we expected the calculation to be ~100x slower. This performance is mostly explained by the target column binning feature, where repeated columns (or those whose probability vectors are close, but not identical) are merged and calculation does not happen for them. Since each column in the target set now has 10 exact copies, it may not be surprising to see that the target axis does not matter as much for the calculations. However, this is an important point because as target databases get larger, *they get more redundant*. So, while we may not see speed improvements as large as we saw here, we will still see time only increase *sublinearly* with the number of targets.

#### Nearest Neighbors

In some applications, you may not want the entire dense similarity matrix, either because calculating the full thing would take too much memory or just generally not be valuable for downstream applications. In these cases, you can keep only the top `n` nearest neighbors (according to the p-value). Doing so requires only setting the `n_nearest` parameter.

In [7]:
p, _, _, _, _, idxs = tomtom(target_pwms, target_pwms, n_nearest=100)
p.shape

(1646, 100)

The return signature should be the same except that an additional matrix of indexes is returned. These indexes are the target indexes for the top `n` returned values.

In [8]:
idxs[0, :10]

array([   0.,  188.,  481.,  900., 1252.,  527.,  314.,  684.,  493.,
        805.])

As you might expect, when calculating the similarity between a set of motifs and itself, the motif itself will be the highest similarity match (hence the 0 as the first index).

#### Approximation Parameters

The implementation in `tangermeme` uses a few appproximation tricks to speed up the algorithm. In general, these tricks involve bins, where the more bins the better the approximation but the more time they take. The default values have been tried in a few situations involving PWMs and seem to work reasonably well, but if you have a non-standard setting you may need to change them. They are:

`n_score_bins`: This is the parameter `t` from the TOMTOM paper. The 100 number is hardcoded into the TOMTOM command-line algorithm, but can be changed. However, in addition to setting a higher value taking more time, you will need more data to support each of the bins. Only set to a higher value when you have enough target columns to support them.

`n_median_bins`: When calculating medians, `tangermeme` avoids sorting and implements a linear-time approximation by binning the range, counting the instances in each bin, and checking what bin corresponds to a cumulative half of all examples. The average value of the examples in the median bin (rather than the middle of the bin) is returned, for higher accuracy.

`n_target_bins`: Each value in the target column (each of the four nucleotides in standard cases) are binned into this number of bins from the minimum observed value to the maximum observed value. Target columns that are identical across each value are merged to speed up the inner loop.